In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


In [8]:
''' load MNIST database '''
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
dataset_path = '../mnist_dataset'

train_dataset = MNIST(dataset_path, transform=transform, train=True, download=True)
valid_dataset = MNIST(dataset_path, transform=transform, train=False, download=True)
test_dataset = MNIST(dataset_path, transform=transform, train=False, download=True)


In [9]:
''' load MNIST dataset by using dataloader'''
batch_size = 64

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                         batch_size=batch_size,
                         shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                         batch_size=1,
                         shuffle=False)


In [10]:
''' test data loader'''
print(len(train_loader))
for batch_idx, (image, label) in enumerate(train_loader):
    if (batch_idx + 1) % 100 == 0:
        print(image.shape, label.shape)


938
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])
torch.Size([64, 1, 28, 28]) torch.Size([64])


In [25]:
''' test implementation '''
# prepare network input 
x_batch, y_batch = iter(train_loader).next()
batch_size, n_chn, hor_dim, ver_dim = x_batch.size()

x = x_batch

# define neural network 
conv1 = nn.Conv2d(1, 6, 5, 1) # 6@24*24
pool1 = nn.MaxPool2d(2) # 6@12*12
conv2 = nn.Conv2d(6, 16, 5, 1) # 16@8*8
pool2 = nn.MaxPool2d(2) # 16@4*4

fc1 = nn.Linear(16 * 4 * 4, 120)
fc2 = nn.Linear(120, 84)
fc3 = nn.Linear(84, 10)

# Conv layer
x = F.relu(conv1(x))
x = pool1(x)
x = F.relu(conv2(x))
x = pool2(x)

# FC layer
x = x.view(-1, 16 * 4 * 4)
x = F.relu(fc1(x))
x = F.relu(fc2(x))
x = fc3(x)

print(x_batch.size())
print(x.size(), y_batch.size())

torch.Size([64, 16, 4, 4])
torch.Size([64, 1, 28, 28])
torch.Size([64, 10]) torch.Size([64])


In [34]:
''' Model class definition '''
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, 1) # 6@24*24
        self.pool1 = nn.MaxPool2d(2) # 6@12*12
        self.conv2 = nn.Conv2d(6, 16, 5, 1) # 16@8*8
        self.pool2 = nn.MaxPool2d(2) # 16@4*4
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Conv layer
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        # FC layer
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN()
print(model)


CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [35]:
''' Training criteria and optimizer definition '''
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
print(criterion, optimizer)

CrossEntropyLoss() Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0001
    weight_decay: 0
)


In [36]:
''' Train network '''
num_epochs = 6
model.train()

for epoch in range(num_epochs):
    loss_avg = 0.
    for image, label in train_loader:
        model_out = model(image)
        loss = criterion(model_out, label)
        loss.backward()
        optimizer.step()
        loss_avg += loss / len(train_loader)
        
    print('Epoch: {:} \tTrain loss: {:.6f}'.format(
        epoch+1, loss_avg))
    

Epoch: 1 	Train loss: 1.051785
Epoch: 2 	Train loss: 0.328588
Epoch: 3 	Train loss: 0.244821
Epoch: 4 	Train loss: 0.250008
Epoch: 5 	Train loss: 0.228589
Epoch: 6 	Train loss: 0.218987


In [37]:
''' Test model '''
test_loss = 0.
accuracy_total = 0
model.eval()
for image, label in test_loader:
    # Evaluate loss
    model_out = model(image)
    loss = criterion(model_out, label)
    test_loss += loss / len(test_loader)
    
    # Evaluate classification accuracy
    _, pred = torch.max(model_out, dim=1)
    accuracy = torch.sum((pred == label).float())
    accuracy_total += accuracy / len(test_loader)
    
print('Test set: Accuracy: {:.2f}, Loss: {:.6f}'.format(
    accuracy_total, test_loss))


Test set: Accuracy: 0.95, Loss: 0.207094
